In [ ]:
#Necessary Modules
import suite2p
import sys
import pandas as pd
import os
import openpyxl
import xml.etree.ElementTree as ET
from pathlib import Path
import math
import re
from classesandfunctions import Animal, getXMLPath, getMag, createListFilesAndParams

In [ ]:
#Read Excel File and Print Column Headers
#If you only want certain animals, change the specific_rows list to the rows of the animals you want

specific_rows = [0,46]
#47
files = pd.read_excel(
    '2023_Dev_Chronic_Summary.xlsx', 
    skiprows = lambda x: x not in specific_rows,
)
files = files.dropna(how='all')
#Prints out excel head for reference, uncomment if you would like
#files.head()

In [ ]:
#Creates an empty list of animals and then ads all animals that aren't missing certain fields to list

listOfAnimals = []

#Drops animals that do not have a filetime as this is used for important parts of the script
#Also drops any animal that has no DOB
for index, row in files.iterrows():
    if((pd.isnull(row['DOB'])) or (pd.isnull(row['Date']))):
        pass
    else:
        dateOfBirth = str(row['DOB'])[:8]
        animal = row['Animal']
        tSeries = row['Tseries']
        magnification = row['Mag']
        age = row['Date_2']
        recordingDate = row['Date']
        listOfAnimals.append(Animal(dateOfBirth,animal,tSeries,magnification,age,recordingDate))

In [ ]:
#Creates a list of folder names without changing original list
tiffParentFolders = []

for animal in listOfAnimals:
    birth = str(animal.recordingDate)
    name = str(animal.name)
    age = str(animal.age)
    
    folder = birth[:8] + "_" + name + "_" + age.lower()
    #Prints each folder name in list, double check list is correct
    #May Comment out if desired
    #print(folder)
    tiffParentFolders.append(folder)

In [ ]:
drivesToSearch = ["/Volumes/NHCAIMG1/", "/Volumes/NHCAIMG/"]
XMLLocs = getXMLPath(listOfAnimals, drivesToSearch)

#Make sure that all the magnifications are correct
for list in XMLLocs:
    if not list:
        continue
    else:
        sameMag = True
        refMag = getMag(list[0])
        stop = len(list)
        for i in range(1,stop):
            curMag = getMag(list[i]) 
            if not math.isclose(refMag,curMag):
                sameMag = False
                break
        if not sameMag:
            print("WARNING: The folder containing the files: \n")
            for file in list:
                print(str(file)[-29:] + "\n")
            print("has different magnifications\nThe full path to the first file is: " + str(list[0]))

In [ ]:
#Setup suite2p with base defaults
from suite2p import run_s2p, default_ops
ops = default_ops()

"""
Input Path to folders that will be searched e.g "G:/" for whole drive on windows
Use forward slashes as the Path method will handle filepaths automatically but only 
allows the use of forward slashes 
"""
folderpaths = ["/Volumes/NHCAIMG1/", "/Volumes/NHCAIMG/"]
#Optionally input save disk if you want all outputs saved to single drive
savediskOrFolder = "/Volumes/NHCAIMG1/"
db = createListFilesAndParams(folderpaths,tiffParentFolders,savediskOrFolder)

In [ ]:
#Run suite2p
for dbi in db:
    opsEnd = run_s2p(ops=ops,db=dbi)